# **Section:** Model Selection

In [1]:
import os
import itertools as it
import warnings
import time
import pickle

import numpy as np

import pandas as pd

%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import cm
import seaborn as sns

import joblib

import pathlib

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.exceptions import DataConversionWarning

from sklearn import metrics

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

import tqdm

from libs.container import Container
from libs.nearest import nearest
from libs.experiment import WithAnotherExperiment, roc, metrics
from libs.precstar import  prec_star

warnings.simplefilter("ignore", category=DataConversionWarning)

/home/jbcabral/miniconda2/envs/howfar/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.ranking module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
PATH = pathlib.Path(os.path.abspath(os.path.dirname("")))

DATA_PATH = PATH / "_data" / "s5k_scaled.pkl.bz2"

COLUMNS_NO_FEATURES = ['id', 'tile', 'cnt', 'ra_k', 'dec_k', 'vs_type', 'vs_catalog', 'cls'] 

In [3]:
sample = pd.read_pickle(DATA_PATH)

# the features
X_columns = [c for c in sample.columns if c not in COLUMNS_NO_FEATURES]
y_column = "cls"

sample[X_columns] =  sample[X_columns].astype(np.float32)

data = Container({k: v for k, v in sample.groupby("tile") if k in ["b234", "b360", "b278", "b261"]})

del sample

In [4]:
def score_func(y, y_prob, **kwargs):
    prec, rec, thr = metrics.precision_recall_curve(
            y, y_prob, sample_weight=None)
    idx = nearest(array=rec, value=.9)
    return prec[idx]


def grid_search(data, estimator, param_grid):
    print(f"Running {type(estimator)}")
    clf = GridSearchCV(
        estimator, 
        param_grid, 
        cv=5, scoring=metrics.make_scorer(score_func, needs_proba=True), n_jobs=-1)

    X, y = data[X_columns].values, data.cls.values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)

    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

    return clf

## SVM-RBF

In [5]:
gamma_range = np.logspace(-9, 3, 13)
gamma_range

array([1.e-09, 1.e-08, 1.e-07, 1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02,
       1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])

In [6]:
%%time
svc_rbf = grid_search(
    data=data.b278, 
    estimator=SVC(),
    param_grid=[{
        'kernel': ['rbf'], 
        'C': [1, 10, 20, 30, 34, 50, 100, 1000], 
        "gamma": gamma_range, 
        "probability": [True]}])

Running <class 'sklearn.svm._classes.SVC'>
Best parameters set found on development set:

{'C': 34, 'gamma': 0.01, 'kernel': 'rbf', 'probability': True}

Grid scores on development set:

0.544 (+/-0.104) for {'C': 1, 'gamma': 1e-09, 'kernel': 'rbf', 'probability': True}
0.543 (+/-0.181) for {'C': 1, 'gamma': 1e-08, 'kernel': 'rbf', 'probability': True}
0.533 (+/-0.154) for {'C': 1, 'gamma': 1e-07, 'kernel': 'rbf', 'probability': True}
0.541 (+/-0.165) for {'C': 1, 'gamma': 1e-06, 'kernel': 'rbf', 'probability': True}
0.540 (+/-0.162) for {'C': 1, 'gamma': 1e-05, 'kernel': 'rbf', 'probability': True}
0.545 (+/-0.174) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf', 'probability': True}
0.635 (+/-0.133) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf', 'probability': True}
0.689 (+/-0.208) for {'C': 1, 'gamma': 0.01, 'kernel': 'rbf', 'probability': True}
0.643 (+/-0.102) for {'C': 1, 'gamma': 0.1, 'kernel': 'rbf', 'probability': True}
0.072 (+/-0.000) for {'C': 1, 'gamma': 1.0, 'kernel': 'rbf'

## SVM-Linear 

In [8]:
%%time

Cs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 34, 50]

svc_linear = grid_search(
    data=data.b278, 
    estimator=SVC(probability=True),
    param_grid=[{'kernel': ['linear'], 'C': Cs, "probability": [True]}])


Running <class 'sklearn.svm._classes.SVC'>
Best parameters set found on development set:

{'C': 4, 'kernel': 'linear', 'probability': True}

Grid scores on development set:

0.643 (+/-0.231) for {'C': 1, 'kernel': 'linear', 'probability': True}
0.622 (+/-0.288) for {'C': 2, 'kernel': 'linear', 'probability': True}
0.631 (+/-0.290) for {'C': 3, 'kernel': 'linear', 'probability': True}
0.652 (+/-0.279) for {'C': 4, 'kernel': 'linear', 'probability': True}
0.645 (+/-0.283) for {'C': 5, 'kernel': 'linear', 'probability': True}
0.637 (+/-0.289) for {'C': 6, 'kernel': 'linear', 'probability': True}
0.639 (+/-0.288) for {'C': 7, 'kernel': 'linear', 'probability': True}
0.632 (+/-0.282) for {'C': 8, 'kernel': 'linear', 'probability': True}
0.634 (+/-0.265) for {'C': 9, 'kernel': 'linear', 'probability': True}
0.632 (+/-0.269) for {'C': 10, 'kernel': 'linear', 'probability': True}
0.639 (+/-0.261) for {'C': 20, 'kernel': 'linear', 'probability': True}
0.645 (+/-0.260) for {'C': 30, 'kernel': 'l

## KNN

In [9]:
k_range = np.arange(1, 30)
k_range

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [10]:
%%time
knn = grid_search(
    data=data.b278, 
    estimator=KNeighborsClassifier(),
    param_grid=[{
        "weights": ['uniform', 'distance'], 
        "algorithm": ['auto', 'ball_tree', 'kd_tree', 'brute'],
        "p": [1, 2, 3],
        "n_neighbors": k_range}])

Running <class 'sklearn.neighbors._classification.KNeighborsClassifier'>
Best parameters set found on development set:

{'algorithm': 'auto', 'n_neighbors': 2, 'p': 1, 'weights': 'distance'}

Grid scores on development set:

0.399 (+/-0.785) for {'algorithm': 'auto', 'n_neighbors': 1, 'p': 1, 'weights': 'uniform'}
0.399 (+/-0.785) for {'algorithm': 'auto', 'n_neighbors': 1, 'p': 1, 'weights': 'distance'}
0.553 (+/-0.775) for {'algorithm': 'auto', 'n_neighbors': 1, 'p': 2, 'weights': 'uniform'}
0.553 (+/-0.775) for {'algorithm': 'auto', 'n_neighbors': 1, 'p': 2, 'weights': 'distance'}
0.368 (+/-0.709) for {'algorithm': 'auto', 'n_neighbors': 1, 'p': 3, 'weights': 'uniform'}
0.368 (+/-0.709) for {'algorithm': 'auto', 'n_neighbors': 1, 'p': 3, 'weights': 'distance'}
0.790 (+/-0.031) for {'algorithm': 'auto', 'n_neighbors': 2, 'p': 1, 'weights': 'uniform'}
0.792 (+/-0.029) for {'algorithm': 'auto', 'n_neighbors': 2, 'p': 1, 'weights': 'distance'}
0.755 (+/-0.058) for {'algorithm': 'auto', 

## Random Forest

In [11]:
%%time

rf = grid_search(
    data=data.b278, 
    estimator=RandomForestClassifier(),
    param_grid=[{
        'max_features': ['auto', 'sqrt', "log2", None, 0.2, 0.5], 
        "min_samples_split": [2, 5, 10],
        "n_estimators": [500], 
        "criterion": ["entropy"], 
        "n_jobs": [10]}])

del rf["n_jobs"]

Running <class 'sklearn.ensemble._forest.RandomForestClassifier'>
Best parameters set found on development set:

{'criterion': 'entropy', 'max_features': 'auto', 'min_samples_split': 10, 'n_estimators': 500, 'n_jobs': 10}

Grid scores on development set:

0.863 (+/-0.226) for {'criterion': 'entropy', 'max_features': 'auto', 'min_samples_split': 2, 'n_estimators': 500, 'n_jobs': 10}
0.871 (+/-0.183) for {'criterion': 'entropy', 'max_features': 'auto', 'min_samples_split': 5, 'n_estimators': 500, 'n_jobs': 10}
0.871 (+/-0.183) for {'criterion': 'entropy', 'max_features': 'auto', 'min_samples_split': 10, 'n_estimators': 500, 'n_jobs': 10}
0.858 (+/-0.192) for {'criterion': 'entropy', 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 500, 'n_jobs': 10}
0.864 (+/-0.162) for {'criterion': 'entropy', 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 500, 'n_jobs': 10}
0.843 (+/-0.194) for {'criterion': 'entropy', 'max_features': 'sqrt', 'min_samples_split': 10, 'n_

In [12]:
best_params = {
    "rf": rf.best_params_,
    "knn": knn.best_params_,
    "svml": svc_linear.best_params_,
    "svmr": svc_rbf.best_params_
}

joblib.dump(best_params, "_cache/best_params.pkl.bz2", compress=3)

['_cache/best_params.pkl.bz2']

In [13]:
best_params

{'rf': {'criterion': 'entropy',
  'max_features': 'auto',
  'min_samples_split': 10,
  'n_estimators': 500,
  'n_jobs': 10},
 'knn': {'algorithm': 'auto', 'n_neighbors': 2, 'p': 1, 'weights': 'distance'},
 'svml': {'C': 4, 'kernel': 'linear', 'probability': True},
 'svmr': {'C': 34, 'gamma': 0.01, 'kernel': 'rbf', 'probability': True}}

In [14]:
import datetime
datetime.datetime.now()

datetime.datetime(2020, 1, 18, 17, 43, 48, 821121)